In [ ]:
#########
#Matthew
#20250102

In [ ]:
#指定目標資料夾的路徑
#影片輸出地
output_foider=""
#輸入全域地址，影片的儲存地
target_folder = "/Users/matthew/Documents/lab/code/Deeplabcut/test"

In [ ]:
#讀取 CSV 檔案
import csv
data=[]
try:
    with open('list.csv', 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            for k in row.split():
                data.append(k)
except:
    print("沒有資料，要跑校正"）

In [ ]:
#以下校正影像
import os
import subprocess

# 定義輸入與輸出資料夾
input_folder = target_folder
output_folder = output_foider

# 確保輸出資料夾存在
os.makedirs(output_folder, exist_ok=True)

# 遍歷所有 MP4 檔案
for filename in os.listdir(input_folder):
    if filename.endswith(".mp4"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        
        # 構造 FFmpeg 命令
        command = [
            "ffmpeg", "-i", input_path,
            "-vf", "lenscorrection=kx="+str(data[0])+":ky="+str(data[1]),
            output_path
        ]
        
        # 執行 FFmpeg 命令
        print(f"Processing {input_path} -> {output_path}")
        subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
print("All videos processed.")

In [ ]:
#以下為影像校正的參數設定
import numpy as np
import cv2 as cv
import glob
import os
import csv

# 進入該資料夾
os.chdir(target_folder)

# 驗證目前的工作目錄
print("目前工作目錄:", os.getcwd())

In [ ]:
data=[]
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('*.jpg')

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)


cv.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

k1 = dist[0][0]
k2 = dist[0][1]

mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error

print( "total error: {}".format(mean_error/len(objpoints)) )
data.append(k1)
data.append(k2)

with open('data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(data)